# Predicting next event: using only activity attributes

Required library

In [19]:
import pandas as pd
import numpy as np

# lazy loading and computation of data
import dask
import dask.dataframe as dd
import dask.array as da

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

import collections


## Preprocessing the dataset

In [11]:
csv_file = "data/financial_log.csv"
txt_file = "data/financial_log.txt"
delimiter = ":||:"
end_word = "[EOC]"
ddf = dd.read_csv(csv_file)

In [12]:
ddf.head(10)

,Case ID,Activity,Resource,Complete Timestamp,Variant,Variant index,lifecycle:transition,concept:name,AMOUNT_REQ
0,173688,A_SUBMITTED-COMPLETE,112.0,2011-10-01 08:38:44.546,Variant 613,613,COMPLETE,A_SUBMITTED,20000
1,173688,A_PARTLYSUBMITTED-COMPLETE,112.0,2011-10-01 08:38:44.880,Variant 613,613,COMPLETE,A_PARTLYSUBMITTED,20000
2,173688,A_PREACCEPTED-COMPLETE,112.0,2011-10-01 08:39:37.906,Variant 613,613,COMPLETE,A_PREACCEPTED,20000
3,173688,W_Completeren aanvraag-SCHEDULE,112.0,2011-10-01 08:39:38.875,Variant 613,613,SCHEDULE,W_Completeren aanvraag,20000
4,173688,W_Completeren aanvraag-START,NaN,2011-10-01 19:36:46.437,Variant 613,613,START,W_Completeren aanvraag,20000
5,173688,A_ACCEPTED-COMPLETE,10862.0,2011-10-01 19:42:43.308,Variant 613,613,COMPLETE,A_ACCEPTED,20000
6,173688,O_SELECTED-COMPLETE,10862.0,2011-10-01 19:45:09.243,Variant 613,613,COMPLETE,O_SELECTED,20000
7,173688,A_FINALIZED-COMPLETE,10862.0,2011-10-01 19:45:09.243,Variant 613,613,COMPLETE,A_FINALIZED,20000
8,173688,O_CREATED-COMPLETE,10862.0,2011-10-01 19:45:11.197,Variant 613,613,COMPLETE,O_CREATED,20000
9,173688,O_SENT-COMPLETE,10862.0,2011-10-01 19:45:11.380,Variant 613,613,COMPLETE,O_SENT,20000


In [13]:
def addEOC(df, EOC):
    for index, row in df.iterrows():
        row['Activity'] = row['Activity'] + EOC
    pass

def output_to_txt(df):
    length = 0
    f = open(txt_file, "w")
    for index, row in df.iterrows():
        if (length<=len(row['Activity'])):
            length = len(row['Activity'])
            max_in = index
        listToStr = ':||:'.join(map(str, row['Activity']))
        listToStr = listToStr.replace(" ","")
        f.write(listToStr)
        f.write("\n")
    f.close()
    return (length, max_in)


In [14]:
case_grouped = ddf.groupby('Case ID')['Activity'].apply(list,meta=('Activity', 'object')).compute().to_frame()
addEOC(case_grouped, [end_word])
output_to_txt(case_grouped)


(176, 185548)

## Word Embedding

In [28]:

def _read_words(filename):
    with open(filename) as f: 
        return f.read().replace("\n", " ").replace(delimiter, " ").split()

def _build_vocab(filename):
  data = _read_words(filename)

  counter = collections.Counter(data)
  count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

  words, _ = list(zip(*count_pairs))
  word_to_id = dict(zip(words, range(len(words))))

#   # Zippo's test
#   import csv
#   w = csv.writer(open("data/output.csv", "w"))
#   for key, val in word_to_id.items():
#      w.writerow([key, val])

  return words



In [29]:
_build_vocab(txt_file)

('W_Completerenaanvraag-COMPLETE',
 'W_Completerenaanvraag-START',
 'W_Nabellenoffertes-COMPLETE',
 'W_Nabellenoffertes-START',
 'A_PARTLYSUBMITTED-COMPLETE',
 'A_SUBMITTED-COMPLETE',
 '[EOC]',
 'W_Nabellenincompletedossiers-COMPLETE',
 'W_Nabellenincompletedossiers-START',
 'W_Validerenaanvraag-COMPLETE',
 'W_Validerenaanvraag-START',
 'A_DECLINED-COMPLETE',
 'W_Completerenaanvraag-SCHEDULE',
 'A_PREACCEPTED-COMPLETE',
 'O_CREATED-COMPLETE',
 'O_SELECTED-COMPLETE',
 'O_SENT-COMPLETE',
 'W_Nabellenoffertes-SCHEDULE',
 'W_Afhandelenleads-COMPLETE',
 'W_Afhandelenleads-START',
 'A_ACCEPTED-COMPLETE',
 'W_Validerenaanvraag-SCHEDULE',
 'A_FINALIZED-COMPLETE',
 'W_Afhandelenleads-SCHEDULE',
 'O_CANCELLED-COMPLETE',
 'O_SENT_BACK-COMPLETE',
 'A_CANCELLED-COMPLETE',
 'W_Nabellenincompletedossiers-SCHEDULE',
 'A_ACTIVATED-COMPLETE',
 'A_APPROVED-COMPLETE',
 'A_REGISTERED-COMPLETE',
 'O_ACCEPTED-COMPLETE',
 'O_DECLINED-COMPLETE',
 'W_Beoordelenfraude-COMPLETE',
 'W_Beoordelenfraude-START',
 'W_

In [16]:
# We create a custom standardization function
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


In [17]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(40, 4, input_length=180))


In [18]:
batch_size = 1024
seed = 123
train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='training', seed=seed)
val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='validation', seed=seed)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'aclImdb/train'